# Zurich Instruments LabOne Python API Example
# Connect to a Zurich Instruments HF2 device

Demonstrate how to connect to a Zurich Instruments HF2 device via the Data Server
program.
Create an API session by connecting to a Zurich Instruments
device via the Data Server, ensure the demodulators are enabled and obtain a
single demodulator sample via getSample(). Calculate the sample's RMS
amplitude and add it as a field to the "sample" dictionary.

> Note:
>
> This is intended to be a simple example demonstrating how to connect to a
> Zurich Instruments device from ziPython. In most cases, data acquisition
> should use either ziDAQServer's poll() method or an instance of the
> ziDAQRecorder class, not the getSample() method.

Requirements:
* LabOne Version >= 20.02
* Instruments:
    1 x Zurich Instruments HF2 device

---

In [ ]:
import numpy as np
import zhinst.ziPython

Set up the connection. The connection is always through a session to a
Data Server. The Data Server then connects to the device.

The LabOne Data Server needs to run within the network, either on localhost when
starting LabOne on your local computer or a remote server. (The MFLI/IA devices
have a Data Server running on the device itself, please see the
[user manual](https://docs.zhinst.com/mfli_user_manual/getting_started/device_connectivity_instrument.html)
for detailed explanation.)

In [ ]:
device_id = "dev1234" # Device serial number available on its rear panel.
interface = "USB"

server_host = "localhost"
server_port = 8005
api_level = 1

# Create an API session to the Data Server.
daq = zhinst.ziPython.ziDAQServer(server_host, server_port, api_level)
# Establish a connection between Data Server and Device.
daq.connectDevice(device_id, interface)

Enable the demodulator and set the demodulator output rate.

In [ ]:
settings = [
    (f"/{device_id}/demods/0/enable", 1),
    (f"/{device_id}/demods/0/rate", 1.0e3)
]
daq.set(settings)

Obtain one demodulator sample. If the demodulator is not enabled (as
above) then the command will time out: we'll get a RuntimeError showing
that a `ZIAPITimeoutException` occurred.

In [ ]:
sample = daq.getSample(f"/{device_id}/demods/0/sample")

Extract the in-phase and quadrature components of demodulated signal.

In [ ]:
X = sample['x'][0]
Y = sample['y'][0]
print(f"Measured in-phase component:\t {X:.3e} V")
print(f"Measured quadrature component:\t {Y:.3e} V")

Calculate the RMS amplitude and phase of demodulated signal..

In [ ]:
R = np.abs(X + 1j*Y)
Theta = np.arctan2(Y,X)
print(f"Measured RMS amplitude:\t {R:.3e} V")
print(f"Measured signal phase:\t {(180/np.pi)*Theta:.2f} deg")